In [1]:
import pytest
import sys
import os 
import pandas as pd
import importlib
%matplotlib inline
sys.path.append('/workspace')
pd.set_option('display.max_columns', None)  
pd.set_option('display.max_rows', None)  

In [2]:
from modules.Pipelines.FlowsPipe import FlowsPipe
from modules.Pipelines.ERCsAggregatedPipe import ERCsAggregatedPipe
from modules.Pipelines.ERCsCategoriesPipe import ERCsCategoriesPipe
from modules.Transformers.Aggregators.NetworkDataMetricsAggregator import NetworkDataMetricsAggregator
from modules.Transformers.Aggregators.NetworkDataXDayAverage import NetworkDataXDayAverage
from modules.Visualizers.ERCs.LineChartERCsAggregateMetric import LineChartERCsAggregateMetric
from modules.Visualizers.MarketShareAssetsByMetric import MarketShareAssetsByMetric
from modules.Visualizers.Table import Table
from modules.Visualizers.LineChart import LineChart
from modules.Transformers.Rankers.RankMetricByAssets import RankMetricByAssets

In [3]:
api_key = 'KKzV6V2DTY87v3m1dGZu'
u_tokens = ['ant', 'bat', 'cennz', 'ctxc', 'cvc', 'fun', 'link', 'loom','gno', 'gnt', 'icn', 'lrc', 'mana', 'mkr', 'omg', 'pay', 'poly', 'powr', 'ppt', 'qash','rep', 'salt', 'srn', 'wtc', 'zrx']
stablecoins = ['dai', 'gusd', 'tusd', 'usdc', 'pax', 'usdt_eth']
exchange_tokens = ['bnb', 'ht', 'knc','leo_eth']
start = '2017-01-01'
metrics = ['AdrActCnt', 'AdrBal1in1BCnt','AdrBalUSD10Cnt','CapMrktCurUSD', 'CapRealUSD','IssTotNtv', 'TxCnt', 'TxTfrCnt', 'TxTfrValAdjUSD', 'VtyDayRet30d']
u_pipe = ERCsCategoriesPipe(api_key=api_key, ERCs=u_tokens, metrics=metrics, start=start, aggregate_ERCs=True, aggregated_assets_name='utility-tokens')
df =u_pipe.run_pipeline().get_data()
stablecoins_pipe = ERCsCategoriesPipe(api_key=api_key, ERCs=stablecoins, metrics=metrics, start=start, aggregate_ERCs=True, aggregated_assets_name='stablecoins')
stablecoins_df =stablecoins_pipe.run_pipeline().get_data()
exchange_tokens_pipe = ERCsCategoriesPipe(api_key=api_key, ERCs=exchange_tokens, metrics=metrics, start=start, aggregate_ERCs=True, aggregated_assets_name='exchange-tokens')
exchange_tokens_df = exchange_tokens_pipe.run_pipeline().get_data()

In [4]:
utility_token_charts = [
   LineChart(title='Market Caps', y_axis_columns=['utility-tokens.CapMrktCurUSD']), 
   LineChart(title='Realized Caps', y_axis_columns=['utility-tokens.CapRealUSD']),
   LineChart(title='Transfer Val', y_axis_columns=['utility-tokens.TxTfrValAdjUSD'])
] 

stablecoin_charts = [
   LineChart(title='Market Caps', y_axis_columns=['stablecoins.CapMrktCurUSD']), 
   LineChart(title='Realized Caps', y_axis_columns=['stablecoins.CapRealUSD']),
   LineChart(title='Transfer Val', y_axis_columns=['stablecoins.TxTfrValAdjUSD'])
] 

exchange_tokens_charts = [
   LineChart(title='Market Caps', y_axis_columns=['exchange-tokens.CapMrktCurUSD']), 
   LineChart(title='Realized Caps', y_axis_columns=['exchange-tokens.CapRealUSD']),
   LineChart(title='Transfer Val', y_axis_columns=['exchange-tokens.TxTfrValAdjUSD'])
] 

charts = [LineChart(title=column, y_axis_columns=[column]) for column in df.columns] 


In [5]:
def concat_dataframes(dfs:list):
    result = pd.DataFrame()
    for df in dfs:
        result = pd.concat([result, df], axis=1, sort=True)
    return result  

In [6]:
combined = concat_dataframes([df, stablecoins_df, exchange_tokens_df])

In [7]:
combined.columns

Index(['utility-tokens.AdrActCnt', 'utility-tokens.TxTfrCnt',
       'utility-tokens.IssTotNtv', 'utility-tokens.TxTfrValAdjUSD',
       'utility-tokens.CapMrktCurUSD', 'utility-tokens.CapRealUSD',
       'utility-tokens.TxCnt', 'utility-tokens.VtyDayRet30d',
       'utility-tokens.AdrBalUSD10Cnt', 'utility-tokens.AdrBal1in1BCnt',
       'stablecoins.AdrActCnt', 'stablecoins.TxTfrCnt',
       'stablecoins.IssTotNtv', 'stablecoins.TxTfrValAdjUSD',
       'stablecoins.CapMrktCurUSD', 'stablecoins.CapRealUSD',
       'stablecoins.TxCnt', 'stablecoins.VtyDayRet30d',
       'stablecoins.AdrBalUSD10Cnt', 'stablecoins.AdrBal1in1BCnt',
       'exchange-tokens.AdrActCnt', 'exchange-tokens.TxTfrCnt',
       'exchange-tokens.IssTotNtv', 'exchange-tokens.TxTfrValAdjUSD',
       'exchange-tokens.CapMrktCurUSD', 'exchange-tokens.CapRealUSD',
       'exchange-tokens.TxCnt', 'exchange-tokens.VtyDayRet30d',
       'exchange-tokens.AdrBalUSD10Cnt', 'exchange-tokens.AdrBal1in1BCnt'],
      dtype='object'

In [13]:
categories = ['utility-tokens', 'stablecoins', 'exchange-tokens']
LineChart(title='Share of Market Cap by Token Category', y_axis_columns=[category + '.CapMrktCurUSD' for category in categories]).create_visual(combined).fig.show()
MarketShareAssetsByMetric(title='Share of Market Cap by Token Category', metric='CapMrktCurUSD', assets=categories).create_visual(combined).fig.show()

In [9]:

[chart.create_visual(df).fig.show() for chart in utility_token_charts]
[chart.create_visual(stablecoins_df).fig.show() for chart in stablecoin_charts]
[chart.create_visual(exchange_tokens_df).fig.show() for chart in exchange_tokens_charts]


[None, None, None]

In [10]:
api_key = 'KKzV6V2DTY87v3m1dGZu'
u_tokens = ['ant', 'bat', 'cennz', 'ctxc', 'cvc', 'fun', 'link', 'loom','gno', 'gnt', 'icn', 'knc', 'lrc', 'mana', 'mkr', 'omg', 'pay', 'poly', 'powr', 'ppt', 'qash','rep', 'salt', 'srn', 'veri', 'wtc', 'zrx']
stablecoins = ['dai', 'gusd', 'tusd', 'usdc', 'pax', 'usdt_eth']
exchange_tokens = ['bnb', 'ht', 'leo_eth']
start = '2017-01-01'
metrics = ['AdrActCnt', 'AdrBal1in1BCnt','AdrBalUSD10Cnt','CapMrktCurUSD', 'CapRealUSD','IssTotNtv', 'TxCnt', 'TxTfrCnt', 'TxTfrValAdjUSD', 'VtyDayRet30d']

In [11]:
stablecoins_indiv_pipe = ERCsCategoriesPipe(api_key=api_key, ERCs=stablecoins, metrics=metrics, start=start, aggregate_ERCs=False)
stablecoins_indiv_df = stablecoins_indiv_pipe.run_pipeline().get_data()

In [12]:
MarketShareAssetsByMetric(title='Stablecoins Market Cap Market Share', metric='CapMrktCurUSD', assets=stablecoins).create_visual(combined).fig.show()

KeyError: 'dai.CapMrktCurUSD'